In [6]:
import requests
import json

In [ ]:
# 利用 requests 對 API 來源發送一個請求
url = 'https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json'
res = requests.get(url)

# 將請求回應的內容存成一個字串格式
text = res.text

# json格式的字串解析成字典或列表
data = json.loads(text)

print(data)

In [36]:
# 計算各縣市藥局數量
print(len(data['features']))
data['features'][0].keys() # ['type', 'properties', 'geometry']
data['features'][0]['properties'] # 基本資料在properties裡

length = len(data['features'])
med_count = {}
for i in range(length):
    county = data['features'][i]['properties']['county']
    if county not in med_count:
        med_count[county] = 0
    med_count[county] += 1

med_count_sort = sorted(med_count.items(), key=lambda x: x[1], reverse=True) # return list
print(med_count_sort)

3293
[('新北市', 501), ('高雄市', 422), ('臺中市', 420), ('臺北市', 339), ('臺南市', 271), ('桃園市', 264), ('彰化縣', 179), ('屏東縣', 140), ('雲林縣', 129), ('嘉義縣', 84), ('宜蘭縣', 76), ('嘉義市', 67), ('南投縣', 67), ('基隆市', 57), ('苗栗縣', 56), ('', 49), ('新竹縣', 46), ('花蓮縣', 46), ('新竹市', 39), ('臺東縣', 23), ('澎湖縣', 11), ('金門縣', 6), ('連江縣', 1)]
